In [1]:
import dask
import os
import datetime
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import importlib

from letkf_forecasting import analyse_results, letkf_io

In [2]:
import re

In [3]:
importlib.reload(analyse_results)
importlib.reload(letkf_io)

base_folder = '/a2/uaren/travis'

file_name = 'all_days'

decimals = 2

runs = [ 'owp_opt', 'persistence', 'opt_flow', 'wrf_no_div', 'wrf_mean', 'radiosonde']
# runs = ['wrf_mean']

In [4]:
save_directory = "/home2/travis/python_code/letkf_forecasting/tables/"

In [5]:
horizons = [15, 30, 45, 60]

In [6]:
legend_dict = {'opt_flow': 'Opt.~Flow', 
               'opt_flow_with_div': 'Opt. Flow w/ Div.',
               'wrf_no_div': 'NWP Winds',
               'wrf': 'NWP w/ Div.',
               'owp_opt': 'ANOC',                                                                                
               'persistence': 'Persis.',
               'radiosonde': 'Radiosonde',
               'wrf_mean': 'NWP Avg.~Winds',
               'ens_member': 'Ens.~Member'} 

In [7]:
rmse = pd.DataFrame(index=horizons, columns=runs)
rmse.index.name = 'Horizon'
correlation = rmse.copy()
bias = rmse.copy()
truth_sd = rmse.copy()
for run_name in runs:
    results_folder_path = os.path.join(                                          
        base_folder,                                                               
        'results',                                                               
        'multi_day_error',
        'third_set',                                                             
        run_name)
    stat_name = 'rmse'
    file_path = os.path.join(results_folder_path,
                             f'{stat_name}.h5')
    rmse[run_name] = pd.read_hdf(file_path, stat_name)
    
    stat_name = 'bias'
    file_path = os.path.join(results_folder_path,
                             f'{stat_name}.h5')
    bias[run_name] = pd.read_hdf(file_path, stat_name)
    
    stat_name = 'correlation'
    file_path = os.path.join(results_folder_path,
                             f'{stat_name}.h5')
    correlation[run_name] = pd.read_hdf(file_path, stat_name)
    
    stat_name = 'truth_sd'
    file_path = os.path.join(results_folder_path,
                             f'{stat_name}.h5')
    truth_sd[run_name] = pd.read_hdf(file_path, stat_name)

In [8]:
peices = [rmse, correlation, bias]
combined = pd.concat(peices, axis=0,
                     keys=['RMSE', 'Corr.', 'Bias'])
combined = combined.rename(columns=legend_dict)

In [9]:
def is_empty(str):
    return str != ''

In [10]:
def format_table(text, header_num=5, footer_num=2):
    text = text.split(' ')
    text = list(filter(is_empty, text))
    text = ' '.join(text)
    split_text = text.split('\n')
    split_titles2 = split_text[2]
    removed = split_titles2[-2:]
    split_titles2 = split_titles2[:-2]
    split_titles2 = split_titles2.split('&')
    for count, this in enumerate(split_titles2):
        if len(this) > 2:
            this = this[0] + '{' + this[1:-1] + '}' + this[-1]
            split_titles2[count] = this
    split_text[2] = '&'.join(split_titles2) + removed
#     split_titles3 = split_text[3].split('&')
#             split_titles2[num] = title[:-8]
#             split_titles3[num] = title[-9:]
#     print(split_titles2)
#     return split_text[2]
#     split_titles2 = '&'.join(split_titles2)
#     split_titles3 = '&'.join(split_titles3)
#     split_text[2] = split_titles2
#     split_text[3] = split_titles3
    
    for line_num, line in enumerate(split_text[header_num:-footer_num - 1]):
        split_line = line.split(' ')
        if split_line[0] == 'Corr.':
            Corr = True
        elif split_line[0] != '':
            Corr = False
        num_slice = slice(4, None, 2)
        numbers_str = split_line[num_slice]
        numbers = np.array(
            split_line[num_slice],
            dtype='float')
        if Corr:
            best_num = numbers.max()
        else:
            best_num = numbers[np.abs(numbers).argmin()]
        argmins = np.where(numbers == best_num)[0]
#         numbers = list(numbers.astype('str'))
        for argmin in argmins:
            numbers_str[argmin] = '\\B ' + numbers_str[argmin]
        split_line[num_slice] = numbers_str
        split_text[header_num + line_num] = ' '.join(split_line)
    
#     for count in range(hor_num - 1):
#         split_line.insert(((count + 1)*run_num)*2 + 2 + count, '&')
#     split_line

    
    return '\n'.join(split_text)

In [11]:
column_format = 'll' + 'S[table-format=-1.3]' * len(runs)
text = combined.round(decimals=decimals).to_latex(column_format=column_format)
text2 = format_table(text)
text2 = re.sub('\\\\textasciitilde', '~', text2, count=5)
print(text2)

\begin{tabular}{llS[table-format=-1.3]S[table-format=-1.3]S[table-format=-1.3]S[table-format=-1.3]S[table-format=-1.3]S[table-format=-1.3]}
\toprule
 & & {ANOC} & {Persis.} & {Opt.~Flow} & {NWP Winds} & {NWP Avg.~Winds} & {Radiosonde} \\
{} & Horizon & & & & & & \\
\midrule
RMSE & 15 & \B 0.11 & 0.12 & \B 0.11 & 0.12 & 0.12 & 0.14 \\
 & 30 & \B 0.12 & 0.14 & 0.13 & 0.14 & 0.14 & 0.15 \\
 & 45 & \B 0.13 & 0.15 & 0.15 & 0.15 & 0.15 & 0.16 \\
 & 60 & \B 0.13 & 0.16 & 0.15 & 0.15 & 0.15 & 0.16 \\
Corr. & 15 & \B 0.88 & 0.84 & 0.87 & 0.85 & 0.86 & 0.80 \\
 & 30 & \B 0.84 & 0.79 & 0.82 & 0.79 & 0.79 & 0.75 \\
 & 45 & \B 0.83 & 0.76 & 0.78 & 0.77 & 0.76 & 0.73 \\
 & 60 & \B 0.81 & 0.74 & 0.75 & 0.76 & 0.75 & 0.71 \\
Bias & 15 & \B -0.00 & \B -0.00 & \B -0.00 & -0.01 & -0.01 & -0.01 \\
 & 30 & \B -0.00 & \B -0.00 & \B -0.00 & -0.02 & -0.02 & -0.01 \\
 & 45 & \B -0.01 & \B -0.01 & \B -0.01 & -0.02 & -0.02 & -0.02 \\
 & 60 & \B -0.01 & \B -0.01 & \B -0.01 & -0.02 & -0.02 & -0.02 \\
\bottomrule
\

In [12]:
this_file = os.path.join(save_directory, f'{file_name}_results.tex')
with open(this_file, 'w') as file:
    file.write(text2)

In [13]:
print(text2)

\begin{tabular}{llS[table-format=-1.3]S[table-format=-1.3]S[table-format=-1.3]S[table-format=-1.3]S[table-format=-1.3]S[table-format=-1.3]}
\toprule
 & & {ANOC} & {Persis.} & {Opt.~Flow} & {NWP Winds} & {NWP Avg.~Winds} & {Radiosonde} \\
{} & Horizon & & & & & & \\
\midrule
RMSE & 15 & \B 0.11 & 0.12 & \B 0.11 & 0.12 & 0.12 & 0.14 \\
 & 30 & \B 0.12 & 0.14 & 0.13 & 0.14 & 0.14 & 0.15 \\
 & 45 & \B 0.13 & 0.15 & 0.15 & 0.15 & 0.15 & 0.16 \\
 & 60 & \B 0.13 & 0.16 & 0.15 & 0.15 & 0.15 & 0.16 \\
Corr. & 15 & \B 0.88 & 0.84 & 0.87 & 0.85 & 0.86 & 0.80 \\
 & 30 & \B 0.84 & 0.79 & 0.82 & 0.79 & 0.79 & 0.75 \\
 & 45 & \B 0.83 & 0.76 & 0.78 & 0.77 & 0.76 & 0.73 \\
 & 60 & \B 0.81 & 0.74 & 0.75 & 0.76 & 0.75 & 0.71 \\
Bias & 15 & \B -0.00 & \B -0.00 & \B -0.00 & -0.01 & -0.01 & -0.01 \\
 & 30 & \B -0.00 & \B -0.00 & \B -0.00 & -0.02 & -0.02 & -0.01 \\
 & 45 & \B -0.01 & \B -0.01 & \B -0.01 & -0.02 & -0.02 & -0.02 \\
 & 60 & \B -0.01 & \B -0.01 & \B -0.01 & -0.02 & -0.02 & -0.02 \\
\bottomrule
\

In [14]:
this_file

'/home2/travis/python_code/letkf_forecasting/tables/all_days_results.tex'

# Skill Score table

In [15]:
def format_table_SS(text, header_num=4, footer_num=2):
    text = text.split(' ')
    text = list(filter(is_empty, text))
    text = ' '.join(text)
    split_text = text.split('\n')
    hor = split_text[3]
    hor = hor.split('&')[0]
    split_text.pop(3)
    split_titles2 = split_text[2]
    removed = split_titles2[-2:]
    split_titles2 = split_titles2[:-2]
    split_titles2 = split_titles2.split('&')
    split_titles2[0] = hor
    for count, this in enumerate(split_titles2):
        if len(this) > 2:
            if this[0] == ' ':
                this = this[1:]
            if this[-1] == ' ':
                this = this[:-1]
            this = ' {' + this + '} '
            split_titles2[count] = this
#     split_text[2] = '&'.join(split_titles2) + removed
    split_text[2] = '&'.join(split_titles2) + removed
#     split_titles3 = split_text[3].split('&')
#             split_titles2[num] = title[:-8]
#             split_titles3[num] = title[-9:]
#     print(split_titles2)
#     return split_text[2]
#     split_titles2 = '&'.join(split_titles2)
#     split_titles3 = '&'.join(split_titles3)
#     split_text[2] = split_titles2
#     split_text[3] = split_titles3
    
    for line_num, line in enumerate(split_text[header_num:-footer_num - 1]):
        split_line = line.split(' ')
        num_slice = slice(2, None, 2)
        numbers_str = split_line[num_slice]
        numbers = np.array(
            split_line[num_slice],
            dtype='float')
        best_num = numbers.max()
        argmins = np.where(numbers == best_num)[0]
#         numbers = list(numbers.astype('str'))
        for argmin in argmins:
            numbers_str[argmin] = '\\B ' + numbers_str[argmin]
        split_line[num_slice] = numbers_str
        split_text[header_num + line_num] = ' '.join(split_line)
    
#     for count in range(hor_num - 1):
#         split_line.insert(((count + 1)*run_num)*2 + 2 + count, '&')
#     split_line

    
    return '\n'.join(split_text)

In [16]:
SS_per = 1 - rmse[
    ['owp_opt',
     'opt_flow',
     'wrf_no_div']].div(
    rmse['persistence'], axis='index')
SS_per = SS_per.rename(columns=legend_dict)

In [17]:
column_format = 'l' + 'S[table-format=-1.3]' * 3
text = SS_per.round(
    decimals=decimals).to_latex(
    column_format=column_format)
text2 = format_table_SS(text)
text2 = re.sub('\\\\textasciitilde', '~', text2, count=5)
print(text)
print(text2)

\begin{tabular}{lS[table-format=-1.3]S[table-format=-1.3]S[table-format=-1.3]}
\toprule
{} &  ANOC &  Opt.\textasciitildeFlow &  NWP Winds \\
Horizon &       &            &            \\
\midrule
15      &  0.12 &       0.09 &       0.05 \\
30      &  0.14 &       0.07 &       0.01 \\
45      &  0.17 &       0.05 &       0.04 \\
60      &  0.16 &       0.03 &       0.05 \\
\bottomrule
\end{tabular}

\begin{tabular}{lS[table-format=-1.3]S[table-format=-1.3]S[table-format=-1.3]}
\toprule
 {Horizon} & {ANOC} & {Opt.~Flow} & {NWP Winds} \\
\midrule
15 & \B 0.12 & 0.09 & 0.05 \\
30 & \B 0.14 & 0.07 & 0.01 \\
45 & \B 0.17 & 0.05 & 0.04 \\
60 & \B 0.16 & 0.03 & 0.05 \\
\bottomrule
\end{tabular}



In [18]:
this_file = os.path.join(save_directory, f'{file_name}_SS.tex')
with open(this_file, 'w') as file:
    file.write(text2)

In [75]:
file_name

'all_days'

In [24]:
this = '1234567'
print(re.sub('23', '56', this))

1564567


In [13]:
this_file

'/home2/travis/python_code/letkf_forecasting/tables/all_days_results.tex'

In [69]:
combined['Persis.']

       Horizon
RMSE   15         0.123792
       30         0.143389
       45         0.153501
       60         0.157643
Corr.  15         0.843885
       30         0.790749
       45         0.759401
       60         0.743022
Bias   15        -0.002230
       30        -0.004713
       45        -0.006853
       60        -0.009265
Name: Persis., dtype: float64

In [79]:
1 - combined.loc['RMSE'].divide(combined.loc['RMSE']['Persis.'], axis=0)

,BACON,Persis.,Opt. Flow,WRF,WRF Mean,Radiosonde
Horizon,,,,,,
15,0.124201,0.0,0.088876,0.049304,0.060016,-0.119027
30,0.137696,0.0,0.071620,0.009154,0.021082,-0.073548
45,0.173878,0.0,0.053268,0.040620,0.031870,-0.035722
60,0.164442,0.0,0.030065,0.047529,0.037172,-0.040449


In [74]:
combined.loc['RMSE']['Persis.']

Horizon
15    0.123792
30    0.143389
45    0.153501
60    0.157643
Name: Persis., dtype: float64

In [72]:
combined.loc['RMSE'] - combined['Persis.'].loc['RMSE']

/home/travis/miniconda3/envs/letkf/lib/python3.6/site-packages/pandas/core/indexes/base.py:3033: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  return this.join(other, how=how, return_indexers=return_indexers)


,BACON,Persis.,Opt. Flow,WRF,WRF Mean,Radiosonde,15,30,45,60
Horizon,,,,,,,,,,
15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
combinedss = combined - combined['Persis.']

ValueError: cannot join with no level specified and no overlapping names

In [21]:
importlib.reload(analyse_results)
importlib.reload(letkf_io)

base_folder = '/a2/uaren/travis'

file_name = 'all_days'

decimals = 3

runs = [ 'owp_opt', 'persistence', 'opt_flow', 'wrf_no_div']
# runs = ['wrf_mean']

In [22]:
save_directory = "/home2/travis/python_code/letkf_forecasting/tables/"

In [23]:
horizons = [15, 30, 45, 60]

In [24]:
legend_dict = {'opt_flow': 'Opt. Flow', 
               'opt_flow_with_div': 'Opt. Flow w/ Div.',
               'wrf_no_div': 'WRF',
               'wrf': 'WRF w/ Div.',
               'owp_opt': 'BACON',                                                                                
               'persistence': 'Persis.',
               'radiosonde': 'Radiosonde',
               'wrf_mean': 'WRF Mean',
               'ens_member': 'Ens. Member'} 

In [49]:
rmse = pd.DataFrame(index=horizons, columns=runs)
rmse.index.name = 'Horizon'
correlation = rmse.copy()
bias = rmse.copy()
truth_sd = rmse.copy()
for run_name in runs:
    results_folder_path = os.path.join(                                          
        base_folder,                                                               
        'results',                                                               
        'multi_day_error',
        'third_set_only_cloudy',                                                             
        run_name)
    stat_name = 'rmse'
    file_path = os.path.join(results_folder_path,
                             f'{stat_name}.h5')
    rmse[run_name] = pd.read_hdf(file_path, stat_name)
    
    stat_name = 'bias'
    file_path = os.path.join(results_folder_path,
                             f'{stat_name}.h5')
    bias[run_name] = pd.read_hdf(file_path, stat_name)
    
    stat_name = 'correlation'
    file_path = os.path.join(results_folder_path,
                             f'{stat_name}.h5')
    correlation[run_name] = pd.read_hdf(file_path, stat_name)
    
    stat_name = 'truth_sd'
    file_path = os.path.join(results_folder_path,
                             f'{stat_name}.h5')
    truth_sd[run_name] = pd.read_hdf(file_path, stat_name)

In [50]:
peices = [rmse, correlation, bias]
combined = pd.concat(peices, axis=0,
                     keys=['RMSE', 'Corr.', 'Bias'])
combined = combined.rename(columns=legend_dict)

In [51]:
def is_empty(str):
    return str != ''

In [52]:
def format_table(text, header_num=5, footer_num=2):
    text = text.split(' ')
    text = list(filter(is_empty, text))
    text = ' '.join(text)
    split_text = text.split('\n')
    split_titles2 = split_text[2].split('&')
    split_titles3 = split_text[3].split('&')
    for num, title in enumerate(split_text[2].split('&')):
        if ' w/ Div.' in title:
            split_titles2[num] = title[:-8]
            split_titles3[num] = title[-9:]
    split_titles2 = '&'.join(split_titles2)
    split_titles3 = '&'.join(split_titles3)
    split_text[2] = split_titles2
    split_text[3] = split_titles3
    
    for line_num, line in enumerate(split_text[header_num:-footer_num - 1]):
        split_line = line.split(' ')
        if split_line[0] == 'Corr.':
            Corr = True
        elif split_line[0] != '':
            Corr = False
        num_slice = slice(4, None, 2)
        numbers_str = split_line[num_slice]
        numbers = np.array(
            split_line[num_slice],
            dtype='float')
        if Corr:
            best_num = numbers.max()
        else:
            best_num = numbers[np.abs(numbers).argmin()]
        argmins = np.where(numbers == best_num)[0]
#         numbers = list(numbers.astype('str'))
        for argmin in argmins:
            numbers_str[argmin] = '\\textbf{' + numbers_str[argmin] + '}'
        split_line[num_slice] = numbers_str
        split_text[header_num + line_num] = ' '.join(split_line)
    
#     for count in range(hor_num - 1):
#         split_line.insert(((count + 1)*run_num)*2 + 2 + count, '&')
#     split_line

    
    return '\n'.join(split_text)

In [53]:
column_format = 'll' + 'c' * len(runs)
text = combined.round(decimals=decimals).to_latex(column_format=column_format)
text2 = format_table(text)
print(text2)

\begin{tabular}{llcccc}
\toprule
 & & BACON & Persis. & Opt. Flow & WRF \\
{} & Horizon & & & & \\
\midrule
RMSE & 15 & \textbf{0.108} & 0.143 & 0.113 & 0.118 \\
 & 30 & \textbf{0.124} & 0.164 & 0.133 & 0.142 \\
 & 45 & \textbf{0.127} & 0.175 & 0.145 & 0.147 \\
 & 60 & \textbf{0.132} & 0.178 & 0.153 & 0.150 \\
Corr. & 15 & \textbf{0.868} & 0.832 & 0.859 & 0.844 \\
 & 30 & \textbf{0.825} & 0.776 & 0.802 & 0.772 \\
 & 45 & \textbf{0.814} & 0.743 & 0.764 & 0.756 \\
 & 60 & \textbf{0.797} & 0.728 & 0.739 & 0.746 \\
Bias & 15 & -0.002 & -0.003 & \textbf{-0.001} & -0.009 \\
 & 30 & -0.007 & \textbf{-0.006} & \textbf{-0.006} & -0.020 \\
 & 45 & -0.011 & \textbf{-0.008} & -0.012 & -0.026 \\
 & 60 & -0.016 & \textbf{-0.011} & -0.014 & -0.031 \\
\bottomrule
\end{tabular}



In [40]:
importlib.reload(analyse_results)
importlib.reload(letkf_io)

base_folder = '/a2/uaren/travis'

file_name = 'all_days'

decimals = 3

runs = [ 'owp_opt', 'persistence', 'opt_flow', 'wrf_no_div']
# runs = ['wrf_mean']

In [41]:
save_directory = "/home2/travis/python_code/letkf_forecasting/tables/"

In [42]:
horizons = [15, 30, 45, 60]

In [43]:
legend_dict = {'opt_flow': 'Opt. Flow', 
               'opt_flow_with_div': 'Opt. Flow w/ Div.',
               'wrf_no_div': 'WRF',
               'wrf': 'WRF w/ Div.',
               'owp_opt': 'BACON',                                                                                
               'persistence': 'Persis.',
               'radiosonde': 'Radiosonde',
               'wrf_mean': 'WRF Mean',
               'ens_member': 'Ens. Member'} 

In [44]:
rmse = pd.DataFrame(index=horizons, columns=runs)
rmse.index.name = 'Horizon'
correlation = rmse.copy()
bias = rmse.copy()
truth_sd = rmse.copy()
for run_name in runs:
    results_folder_path = os.path.join(                                          
        base_folder,                                                               
        'results',                                                               
        'multi_day_error',
        'third_set_only_cloudy_old',                                                             
        run_name)
    stat_name = 'rmse'
    file_path = os.path.join(results_folder_path,
                             f'{stat_name}.h5')
    rmse[run_name] = pd.read_hdf(file_path, stat_name)
    
    stat_name = 'bias'
    file_path = os.path.join(results_folder_path,
                             f'{stat_name}.h5')
    bias[run_name] = pd.read_hdf(file_path, stat_name)
    
    stat_name = 'correlation'
    file_path = os.path.join(results_folder_path,
                             f'{stat_name}.h5')
    correlation[run_name] = pd.read_hdf(file_path, stat_name)
    
    stat_name = 'truth_sd'
    file_path = os.path.join(results_folder_path,
                             f'{stat_name}.h5')
    truth_sd[run_name] = pd.read_hdf(file_path, stat_name)

In [45]:
peices = [rmse, correlation, bias]
combined = pd.concat(peices, axis=0,
                     keys=['RMSE', 'Corr.', 'Bias'])
combined = combined.rename(columns=legend_dict)

In [46]:
def is_empty(str):
    return str != ''

In [47]:
def format_table(text, header_num=5, footer_num=2):
    text = text.split(' ')
    text = list(filter(is_empty, text))
    text = ' '.join(text)
    split_text = text.split('\n')
    split_titles2 = split_text[2].split('&')
    split_titles3 = split_text[3].split('&')
    for num, title in enumerate(split_text[2].split('&')):
        if ' w/ Div.' in title:
            split_titles2[num] = title[:-8]
            split_titles3[num] = title[-9:]
    split_titles2 = '&'.join(split_titles2)
    split_titles3 = '&'.join(split_titles3)
    split_text[2] = split_titles2
    split_text[3] = split_titles3
    
    for line_num, line in enumerate(split_text[header_num:-footer_num - 1]):
        split_line = line.split(' ')
        if split_line[0] == 'Corr.':
            Corr = True
        elif split_line[0] != '':
            Corr = False
        num_slice = slice(4, None, 2)
        numbers_str = split_line[num_slice]
        numbers = np.array(
            split_line[num_slice],
            dtype='float')
        if Corr:
            best_num = numbers.max()
        else:
            best_num = numbers[np.abs(numbers).argmin()]
        argmins = np.where(numbers == best_num)[0]
#         numbers = list(numbers.astype('str'))
        for argmin in argmins:
            numbers_str[argmin] = '\\textbf{' + numbers_str[argmin] + '}'
        split_line[num_slice] = numbers_str
        split_text[header_num + line_num] = ' '.join(split_line)
    
#     for count in range(hor_num - 1):
#         split_line.insert(((count + 1)*run_num)*2 + 2 + count, '&')
#     split_line

    
    return '\n'.join(split_text)

In [48]:
column_format = 'll' + 'c' * len(runs)
text = combined.round(decimals=decimals).to_latex(column_format=column_format)
text2 = format_table(text)
print(text2)

\begin{tabular}{llcccc}
\toprule
 & & BACON & Persis. & Opt. Flow & WRF \\
{} & Horizon & & & & \\
\midrule
RMSE & 15 & \textbf{0.108} & 0.142 & 0.113 & 0.118 \\
 & 30 & \textbf{0.124} & 0.163 & 0.133 & 0.142 \\
 & 45 & \textbf{0.127} & 0.174 & 0.145 & 0.147 \\
 & 60 & \textbf{0.132} & 0.177 & 0.153 & 0.150 \\
Corr. & 15 & \textbf{0.868} & 0.832 & 0.859 & 0.844 \\
 & 30 & \textbf{0.825} & 0.776 & 0.803 & 0.772 \\
 & 45 & \textbf{0.815} & 0.744 & 0.764 & 0.756 \\
 & 60 & \textbf{0.797} & 0.728 & 0.739 & 0.746 \\
Bias & 15 & -0.002 & -0.003 & \textbf{-0.001} & -0.009 \\
 & 30 & -0.008 & \textbf{-0.006} & \textbf{-0.006} & -0.020 \\
 & 45 & -0.012 & \textbf{-0.009} & -0.012 & -0.026 \\
 & 60 & -0.016 & \textbf{-0.011} & -0.015 & -0.031 \\
\bottomrule
\end{tabular}

